In [3]:
import sys
sys.path.append("./")
import numpy as np
from quaos.basic_functions import int_to_bases, bases_to_int,complex_phase_value
from quaos.paulis import (PauliSum, PauliString, Pauli,)
import sympy as sym
from quaos.circuits import Gate, Circuit, Hadamard as H, SUM as CX, PHASE as S
from collections import defaultdict
from sympy.physics.quantum import TensorProduct,Operator
import random
from quaos.hamiltonian import *

In [2]:
P = random_pauli_hamiltonian(2, [2,2], mode='uniform', seed=None)
#P1 = random_pauli_hamiltonian(4, [2,2], mode='uniform', seed=None)
C = Circuit([2,2],gates=[H(0,2)])
print(P)
print(symmetry_cost_function(P, C))

(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x1z1 x1z0 | 0 



NameError: name 'symmetry_cost_function' is not defined

In [87]:
def symmetry_cost_function(P : PauliSum, C : Circuit|Gate):
    P.weights = np.abs(P.weights)
    P1 = C.act(P)
    P1.phase_to_weight()
    P_dict = {}
    for i in range(P.n_paulis()):
        P_dict[str(P[i])] = -np.abs(P.weights[i])
    for i in range(P.n_paulis()):
        if str(P1[i]) in P_dict.keys():
            P_dict[str(P1[i])] += np.around(P1.weights[i].real,decimals=10)
        else:
            P_dict[str(P1[i])] = np.around(P1.weights[i].real,decimals=10)
    cost = sum(abs(value) for value in P_dict.values())
    return(cost)

def symplectic_to_PauliSum(P_sym,weights):
    n_q = int(P_sym.shape[1]/2)
    n_p = int(P_sym.shape[0])
    P = PauliSum([PauliString(np.array(P_sym[i,0:n_q],dtype=np.int32),np.array(P_sym[i,n_q:2*n_q],dtype=np.int32),dimensions=[2 for j in range(n_q)]) for i in range(n_p)],dimensions=[2 for j in range(n_q)], phases=None,weights=weights, standardise=False)
    return(P)

def M_to_cost(M,P,C):
    P_sym = P.symplectic()
    P_c = P_sym @ M.T
    P_c_PS = symplectic_to_PauliSum(P_c,P.weights)
    cost = symmetry_cost_function(P_c_PS, C)
    return(cost)

def symplectic_form(n):
    """Return the standard 2n x 2n symplectic form matrix."""
    I = np.identity(n, dtype=np.uint8)
    O = np.zeros((n, n), dtype=np.uint8)
    return np.block([
        [O, I],
        [I, O]
    ])

def symplectic_dot(u, v, J):
    """Symplectic inner product: u^T J v mod 2"""
    return int(np.dot(u, J @ v) % 2)

def random_isotropic_vector(J):
    """Generate a random vector v such that <v, v> = 0."""
    while True:
        v = np.random.randint(0, 2, size=(J.shape[0],), dtype=np.uint8)
        if symplectic_dot(v, v, J) == 0:
            return v
        
def symplectic_transvection(v, J):
    """Return the matrix T_v(w) = w + <v, w> * v (mod 2)."""
    v = v.reshape(-1, 1)
    return (np.identity(len(v), dtype=np.uint8) + (J @ v) @ v.T) % 2

def update_M(M):
    n = int(M.shape[0]/2)
    J = symplectic_form(n)
    v = random_isotropic_vector(J)
    Mv = symplectic_transvection(v, J)
    M_res = Mv @ M % 2
    return(M_res)

def is_symplectic(M):
    """Check if M^T J M = J over GF(2)."""
    J = symplectic_form(int(M.shape[0]/2))
    left = (M.T @ J @ M) % 2
    return np.array_equal(left, J)


In [88]:
def simulated_annealing(P,C,step_max = 99,T_min = 10**(-4)):
    #weights = P.weights
    P_0 = P.symplectic()
    M_0 = np.identity(2*P.n_qudits())
    cost_0 = M_to_cost(M_0,P,C)
    M_best = M_0
    cost_best = cost_0
    T = 1
    alpha = (T_min)**(1/step_max)
    for i in range(step_max):
        T *= alpha
        M_1 = update_M(M_0)
        cost_1 = M_to_cost(M_1,P,C)
        prob = min(1, np.exp(-(cost_1 - cost_0)/T))
        if prob > np.random.rand():
            M_0 = M_1
            cost_0 = cost_1
        
        if cost_0 < cost_best:
            M_best = M_0
            cost_best = cost_0
        
        if cost_0 ==0:
            return(M_0, symplectic_to_PauliSum(P_0 @ M_0.T,P.weights),cost_0)
    
    return(M_best, symplectic_to_PauliSum(P_0 @ M_best.T,P.weights),cost_best)
    

In [94]:
P = PauliSum(['x0z0 x0z0','x0z0 x0z1','x0z0 x1z0','x0z1 x0z0'],dimensions=[2,2],weights=[1,1,1,1],phases=[0,0,0,0],standardise=False)
print(P)
#weights = P.weights
#P_sym = P.symplectic()
#print(P_sym)
#print(symplectic_to_PauliSum(P_sym,weights))
C = Circuit([2,2],gates=[H(0,2)])
M, P_sym, cost = simulated_annealing(P,C,step_max = 2000)
print(M)
print(P_sym)
print(cost)

(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z1 x0z0 | 0 

[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 

0.0


In [82]:
symmetry_cost_function(P_sym, C)

1.0|x0z0 x0z0 | 0 
1.0|x1z0 x0z1 | 0 
1.0|x0z1 x0z1 | 0 
1.0|x0z0 x0z1 | 0 

(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z1 x0z1 | 0 
(1+0j)|x1z0 x0z1 | 0 
(1+0j)|x0z0 x0z1 | 0 

{'x0z0 x0z0 ': 0.0, 'x1z0 x0z1 ': 0.0, 'x0z1 x0z1 ': 0.0, 'x0z0 x0z1 ': 0.0}


0.0

In [84]:
(10**(-4))**(1/99)

0.9111627561154892

In [7]:
A = np.arange(9).reshape(3,3)
B = np.arange(9).reshape(3,3)
print(A)
print(B.T)
print(A @ B.T)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 3 6]
 [1 4 7]
 [2 5 8]]
[[  5  14  23]
 [ 14  50  86]
 [ 23  86 149]]


In [81]:
import numpy as np

def symplectic_form(n):
    """Return the standard 2n x 2n symplectic form matrix."""
    I = np.identity(n, dtype=np.uint8)
    O = np.zeros((n, n), dtype=np.uint8)
    return np.block([
        [O, I],
        [I, O]
    ])

def symplectic_dot(u, v, J):
    """Symplectic inner product: u^T J v mod 2"""
    return int(np.dot(u, J @ v) % 2)

def symplectic_transvection(v, J):
    """Return the matrix T_v(w) = w + <v, w> * v (mod 2)."""
    v = v.reshape(-1, 1)
    return (np.identity(len(v), dtype=np.uint8) + (J @ v) @ v.T) % 2

def random_isotropic_vector(J):
    """Generate a random vector v such that <v, v> = 0."""
    while True:
        v = np.random.randint(0, 2, size=(J.shape[0],), dtype=np.uint8)
        if symplectic_dot(v, v, J) == 0:
            return v

def generate_random_symplectic(n, num_transvections=None):
    """Generate a random element of Sp(2n, F2) using transvections."""
    J = symplectic_form(n)
    dim = 2 * n
    T = np.identity(dim, dtype=np.uint8)

    if num_transvections is None:
        num_transvections = 2 * dim  # reasonable default

    for _ in range(num_transvections):
        v = random_isotropic_vector(J)
        Tv = symplectic_transvection(v, J)
        T = Tv @ T % 2  # compose

    return T

# Example usage
n = 3  # Sp(6, F2)
M = generate_random_symplectic(n)
print("Random symplectic matrix:")
print(M)

def is_symplectic(M):
    """Check if M^T J M = J over GF(2)."""
    J = symplectic_form(int(M.shape[0]/2))
    left = (M.T @ J @ M) % 2
    return np.array_equal(left, J)

is_symplectic(M)

Random symplectic matrix:
[[0 1 1 0 0 1]
 [1 0 1 1 0 1]
 [0 1 0 1 1 1]
 [0 1 0 0 1 0]
 [0 0 0 1 0 0]
 [0 1 1 1 1 0]]


True

In [6]:
M.shape

(6, 6)